https://data-flair.training/blogs/python-mini-project-speech-emotion-recognition/


dataset: https://drive.google.com/drive/folders/1e4W5LRNDOOHlUNUTzqHM1zpsbLi9kr5f?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
Root = "/content/drive/MyDrive/Ai_Ml_Project/speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust', 'surprised']

In [ ]:
#Load the data and extract features for each sound file
def load_data(test_size=0.25):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Ai_Ml_Project/speech-emotion-recognition-ravdess-data//Actor_*//*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
x_train

array([[-4.77166077e+02,  6.01137733e+01, -3.15369296e+00, ...,
         7.71646097e-04,  1.00887730e-03,  6.25238463e-04],
       [-5.73132751e+02,  3.18506718e+01, -8.54179955e+00, ...,
         1.03158131e-03,  6.40605518e-04,  2.01104325e-04],
       [-6.37922302e+02,  2.80745201e+01, -3.96908331e+00, ...,
         1.30701606e-04,  1.49514613e-04,  1.19012620e-04],
       ...,
       [-5.17250671e+02,  3.25498238e+01, -1.96352844e+01, ...,
         1.20064768e-04,  7.11390458e-05,  4.11658039e-05],
       [-5.88087830e+02,  4.46294594e+01, -1.47748652e+01, ...,
         9.03716063e-05,  1.91249419e-05,  1.12573052e-05],
       [-6.01231934e+02,  4.61873779e+01, -1.52429905e+01, ...,
         1.04361658e-04,  6.08918417e-05,  8.39796194e-05]])

In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

(720, 240)
Features extracted: 180


In [ ]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(450,), learning_rate='adaptive', max_iter=1000)


#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(450,),
              learning_rate='adaptive', max_iter=1000)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

y_pred

array(['calm', 'surprised', 'surprised', 'happy', 'happy', 'calm',
       'disgust', 'calm', 'surprised', 'calm', 'surprised', 'happy',
       'fearful', 'surprised', 'happy', 'surprised', 'disgust', 'calm',
       'disgust', 'happy', 'calm', 'happy', 'disgust', 'fearful',
       'fearful', 'happy', 'calm', 'happy', 'happy', 'fearful', 'calm',
       'happy', 'surprised', 'calm', 'calm', 'surprised', 'calm', 'happy',
       'surprised', 'disgust', 'surprised', 'disgust', 'surprised',
       'calm', 'happy', 'happy', 'surprised', 'happy', 'calm',
       'surprised', 'disgust', 'happy', 'happy', 'disgust', 'calm',
       'happy', 'calm', 'happy', 'happy', 'fearful', 'calm', 'calm',
       'calm', 'disgust', 'disgust', 'calm', 'disgust', 'happy', 'calm',
       'fearful', 'fearful', 'fearful', 'fearful', 'surprised',
       'surprised', 'surprised', 'happy', 'surprised', 'calm', 'fearful',
       'happy', 'fearful', 'happy', 'disgust', 'surprised', 'fearful',
       'disgust', 'fearful', 

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 68.33%


In [ ]:
f1_score(y_test, y_pred,average=None)

array([0.86238532, 0.6       , 0.65882353, 0.57407407, 0.68518519])

In [ ]:
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,calm,calm
1,surprised,surprised
2,surprised,surprised
3,happy,happy
4,fearful,happy
5,calm,calm
6,disgust,disgust
7,calm,calm
8,surprised,surprised
9,calm,calm


In [ ]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [ ]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/Ai_Ml_Project/speech-emotion-recognition-ravdess-data/ndata07.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['calm'], dtype='<U9')